# Jena Reasoning on a simple BOT instance

In order to run Jena using the IJava kernel on Jupyter notebooks, first we need to use some [IJava Magics](https://github.com/SpencerPark/IJava/blob/7f5aa9ab858859a010fd228364b080606399ae60/docs/magics.md#line-magic-2) to load it:



In [1]:
%%loadFromPOM
<dependency>
    <groupId>org.apache.jena</groupId>
    <artifactId>apache-jena-libs</artifactId>
    <type>pom</type>
    <version>3.11.0</version>
</dependency>

In [2]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId> org.apache.cassandra</groupId>
    <artifactId>cassandra-all</artifactId>
    <version>0.8.1</version>

    <exclusions>
      <exclusion> 
        <groupId>org.slf4j</groupId>
        <artifactId>slf4j-log4j12</artifactId>
      </exclusion>
      <exclusion> 
        <groupId>log4j</groupId>
        <artifactId>log4j</artifactId>
      </exclusion>
    </exclusions> 

  </dependency>
</dependencies>

### Setting up Jena 

first, lets import all the Classes we will need for reasoning

In [4]:
import org.apache.jena.reasoner.*;
import org.apache.jena.rdf.model.InfModel;
import org.apache.jena.rdf.model.Statement;
import org.apache.jena.util.*;
import org.apache.jena.rdf.model.*;
import org.apache.jena.ontology.*;
import org.apache.jena.vocabulary.OWL ;
import org.apache.jena.vocabulary.RDF ;
import org.apache.jena.vocabulary.RDFS;

### Load an ABox from the the documentation example
Now,load a simple model that has been created similar to the example in the BOT documentation on https://w3c-lbd-cg.github.io/bot/?tab=0#two-storey-building 

This is provided in the `../data` directory

it consist of just a number of statements:

```
:SiteA a bot:Site ;
    bot:hasBuilding :BuildingA .
    
:BuildingA bot:hasStorey :Storey00 , :Storey01 .

:Storey00 bot:hasSpace :SpaceA , :SpaceB .

:Storey01 bot:hasSpace :SpaceC , :SpaceD .
```

Let's print out what is known abou the :BuildingA



In [8]:
// load a simple instance file from the example az
// https://w3c-lbd-cg.github.io/bot/?tab=0#two-storey-building
/****** the A-BOX *************/
Model simple_house = FileManager.get().loadModel("../data/simple_bot.ttl");
System.out.println ("/***************** facts BEFORE inference ****************/");
Resource Site = simple_house.getResource("https://ldac.org/house#BuildingA");
for (StmtIterator i = ((Model)simple_house).listStatements(new SimpleSelector( Site, null, (RDFNode)null )); i.hasNext();){
    Statement stmt = i.nextStatement();
    System.out.println (PrintUtil.print(stmt));
}

/***************** facts BEFORE inference ****************/
(<https://ldac.org/house#BuildingA> <https://w3id.org/bot#hasStorey> <https://ldac.org/house#Storey01>)
(<https://ldac.org/house#BuildingA> <https://w3id.org/bot#hasStorey> <https://ldac.org/house#Storey00>)


### Load the BOT TBox 
Up to know, these are just the "stupid" facts asserted in the above snipptes. No knowledge about what the Resources actually are is present at present. 

In a second step the TBOX is loaded directly from its source.

A Reasoner is created in that combines TBox and Abox and starts to make inferences

In [6]:
// import the BOT Ontology from the source via HTTP
/****** the T-BOX *************/
Model bot = FileManager.get().loadModel("https://w3id.org/bot#");

// create on OWL reasoner and feed it the bot ontology model
Reasoner reasoner = ReasonerRegistry.getOWLReasoner();
reasoner = reasoner.bindSchema(bot);

//create inferences from 
InfModel infmodel = ModelFactory.createInfModel(reasoner, simple_house);
System.out.println ("\n\n/***************** facts AFTER inference ****************/");
// print out all statements asserted and inferred 
Resource Site = infmodel.getResource("https://ldac.org/house#BuildingA");
for (StmtIterator i = ((Model)infmodel).listStatements(new SimpleSelector( Site, null, (RDFNode)null )); i.hasNext();){
    Statement stmt = i.nextStatement();
    System.out.println (PrintUtil.print(stmt));
}



/***************** facts AFTER inference ****************/
(<https://ldac.org/house#BuildingA> <https://w3id.org/bot#hasStorey> <https://ldac.org/house#Storey01>)
(<https://ldac.org/house#BuildingA> <https://w3id.org/bot#hasStorey> <https://ldac.org/house#Storey00>)
(<https://ldac.org/house#BuildingA> rdf:type owl:Thing)
(<https://ldac.org/house#BuildingA> rdf:type <https://w3id.org/bot#Building>)
(<https://ldac.org/house#BuildingA> <https://w3id.org/bot#containsZone> <https://ldac.org/house#Storey01>)
(<https://ldac.org/house#BuildingA> <https://w3id.org/bot#containsZone> <https://ldac.org/house#Storey00>)
(<https://ldac.org/house#BuildingA> rdf:type <https://w3id.org/bot#Zone>)
(<https://ldac.org/house#BuildingA> rdf:type rdfs:Resource)
(<https://ldac.org/house#BuildingA> <https://w3id.org/bot#containsZone> <https://ldac.org/house#SpaceD>)
(<https://ldac.org/house#BuildingA> <https://w3id.org/bot#containsZone> <https://ldac.org/house#SpaceC>)
(<https://ldac.org/house#BuildingA> <ht

### Extend the BOT ontology with a "connectivity" property

A simple extension is loaded that consists of the following three statements:
```
@prefix bot:  <https://w3id.org/bot#> .
@prefix conn:  <https://ldac.org/connection#> .
@prefix rdfs:  <http://www.w3.org/2000/01/rdf-schema#> .
@prefix owl:  <http://www.w3.org/2002/07/owl#> .

conn:isConnectedTo a owl:SymmetricProperty;
             a owl:TransitiveProperty;
             rdfs:range bot:Zone.
```

The `owl:SymmetricProperty` means that 
- if A is connected to B 
- then it follows that B is connected to A

The `owl:TransitiveProperty` means that
- if A is connected to B
- and B is connected to C
- then A is connected to C

combining both also means 
- C is connected to A

In our ABox, we  a few statements to that end have been added:
```
:SpaceA conn:isConnectedTo :SpaceB .
:SpaceB conn:isConnectedTo :SpaceC .
:Storey00 conn:isConnectedTo :SpaceA .
```

Lets take a look at `:Storey00`


In [9]:
Model conn = FileManager.get().loadModel("../data/conn.ttl");
bot.add(conn);

Reasoner reasoner = ReasonerRegistry.getOWLReasoner();
reasoner = reasoner.bindSchema(bot);

//create inferences from 
InfModel infmodel = ModelFactory.createInfModel(reasoner, simple_house);

//simple_house.write(System.out,"RDF/XML");
Resource Site = infmodel.getResource("https://ldac.org/house#Storey00");
for (StmtIterator i = ((Model)infmodel).listStatements(new SimpleSelector( Site, null, (RDFNode)null )); i.hasNext();){
    Statement stmt = i.nextStatement();
    System.out.println (PrintUtil.print(stmt));
}

(<https://ldac.org/house#Storey00> <https://ldac.org/connection#isConnectedTo> <https://ldac.org/house#SpaceA>)
(<https://ldac.org/house#Storey00> <https://w3id.org/bot#hasSpace> <https://ldac.org/house#SpaceB>)
(<https://ldac.org/house#Storey00> <https://w3id.org/bot#hasSpace> <https://ldac.org/house#SpaceA>)
(<https://ldac.org/house#Storey00> rdf:type <https://w3id.org/bot#Storey>)
(<https://ldac.org/house#Storey00> rdf:type owl:Thing)
(<https://ldac.org/house#Storey00> <https://w3id.org/bot#containsZone> <https://ldac.org/house#SpaceB>)
(<https://ldac.org/house#Storey00> <https://w3id.org/bot#containsZone> <https://ldac.org/house#SpaceA>)
(<https://ldac.org/house#Storey00> rdf:type <https://w3id.org/bot#Zone>)
(<https://ldac.org/house#Storey00> <https://ldac.org/connection#isConnectedTo> <https://ldac.org/house#SpaceB>)
(<https://ldac.org/house#Storey00> <https://ldac.org/connection#isConnectedTo> <https://ldac.org/house#SpaceC>)
(<https://ldac.org/house#Storey00> <https://ldac.org/